In [4]:
# 下载文件。
!wget https://github.com/LC1332/Needy-Haruhi/blob/main/data/Jines.xlsx


--2023-11-05 01:01:17--  https://github.com/LC1332/Needy-Haruhi/blob/main/data/Jines.xlsx
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4583 (4.5K) [text/plain]
Saving to: ‘Jines.xlsx’

Jines.xlsx          100%[===================>]   4.48K  --.-KB/s    in 0s      

2023-11-05 01:01:17 (50.6 MB/s) - ‘Jines.xlsx’ saved [4583/4583]



In [9]:
! pip install pandas

In [13]:
import os
import re
import shutil

import pandas as pd

Jines_file = r"/content/Jines.xlsx"

#读取与查询
Jines = pd.read_excel(Jines_file)
# 查询有选项的内容
pattern = r'(?<=\().*?(?=\))'

# 匹配事件
Title = Jines.loc[(Jines['ParentId (more info)'].str.contains(pattern, regex=True, na=False))]

Attribute_temp = {"Affection": 0, "Stress": 0, "Darkness": 0}


# 把文件转换为txt
def format_output(row):
    # ID
    ParentId = f'{row["ParentId (more info)"]}'
    Category_temp = f'{row["Category"]}'
    Category = sanitize_filename(Category_temp)
    ID = f'{row["Id"]}'

    # 匹配标题
    regex1 = r"\w+(?= \()"
    title = re.search(regex1, ParentId)
    title_str = title.group()

    # 事件
    event_list = []

    # 匹配提问
    match = re.search(r"\(First Part\)", ParentId)
    match2 = re.search(r"\(First Part; end\)", ParentId)
    match3 = re.search(r"\(Third Part\)", ParentId)
    match4 = re.search(r"\(Second Part\)", ParentId)
    match5 = re.search(r"\(Fourth Part\)", ParentId)

    # 数值
    aff = f"Affection: {row['Affection']}"
    str = f"Stress: {row['Stress']}"
    dar = f"Darkness: {row['Darkness']}"

    # 匹配选项以及回复
    choose_time = re.search(r"\d+", ParentId)
    reply_ = re.search(r'(\(.*Option[0-9]+;end\))', ParentId)
    reply_2 = re.search(r'(\(.*Option[0-9]\))', ParentId)

    # 处理提问
    # if match or match2 or match4 or match5 or match3:
    if match or match2 or match3 or match4 or match5:

        Prefix = f'\n## 对话\n### Prefix Category_temp:{Category} ID:{ID}'
        Ame = f"糖糖: {row['BodyCn']}"
        with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
            # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
            line = '\n'.join([Prefix, Ame])

            line_bytes = line.encode('utf-8')
            # 将字节对象写入到文件中
            line_str = line_bytes.decode('utf-8')
            # 将字符串对象写入到文件中
            f.write(line_str)

        return "\n".join([Prefix, Ame])

    # 处理选项
    elif row['Speaker/Action (in blue)'] == 'pi':
        # 跳过数值为空的回复
        try:
            key = f'\n### Option-{choose_time.group()}'
            user = f"User:　{row['BodyCn']}"

            if aff == 'Affection: nan':
                aff = ''
            if str == 'Stress: nan':
                str = ''
            if dar == 'Darkness: nan':
                dar = ''
            value = f"Attribute Change: {aff} {str} {dar}"

            if value == 'Attribute Change:   ':
                value = ''

            with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                line = '\n'.join([key, user, value])

                line_bytes = line.encode('utf-8')
                # 将字节对象写入到文件中
                line_str = line_bytes.decode('utf-8')
                # 将字符串对象写入到文件中
                f.write(line_str)
            return "\n".join([key, user, value])
        except:
            pass

    # 处理选项回复
    elif reply_ or (reply_2 and row['Speaker/Action (in blue)'] == 'ame'):
        try:
            key = f'\nReply：\n糖糖：{row["BodyCn"]}'

            if aff == 'Affection: nan':
                aff = ''
            if str == 'Stress: nan':
                str = ''
            if dar == 'Darkness: nan':
                dar = ''
            value = f"Attribute Change: {aff} {str} {dar}"

            if value == 'Attribute Change:   ':
                value = 'Attribute Change: None'

            if key == '\nReply：\n糖糖：nan':
                with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                    # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                    line = '\n'.join([value])
                    line_bytes = line.encode('utf-8')
                    # 将字节对象写入到文件中
                    line_str = line_bytes.decode('utf-8')
                    # 将字符串对象写入到文件中
                    f.write(line_str)

                return "\n".join([value])

            with open(f'events/{title_str}.txt', 'a+', encoding='utf-8') as f:
                # 使用 join 方法将 Ame, Title_ame, Category 连接成一个字符串，并在每个字段之间添加一个制表符
                line = '\n'.join([key, value])

                line_bytes = line.encode('utf-8')
                # 将字节对象写入到文件中
                line_str = line_bytes.decode('utf-8')
                # 将字符串对象写入到文件中
                f.write(line_str)

            return "\n".join([key, value])
        except:
            pass


ValueError: ignored

In [ ]:
# txt文件转换为jsonl
def parse_to_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = iter(f.readlines())
    dialogs = []
    dialog = {}
    option = {}
    for line in lines:
        line = line.strip()
        if line.startswith("## 对话") or line.startswith("## 对话组"):
            if dialog and option:
                dialog["options"].append(option)
                option = {}
            if dialog:
                dialogs.append(dialog)
            dialog = {"prefix": "", "options": []}
        elif line.startswith("### Prefix") or line.startswith('**Prefix'):
            prefix = next(lines).strip()
            ids, categories = search_in_excel(prefix)
            # print(ids, categories)
            if ids and categories:
                dialog["id"] = ids[0]
                dialog["category"] = categories[0]
            dialog["prefix"] = prefix
        elif line.startswith("### Option") or line.startswith('**Option'):
            if option:
                dialog["options"].append(option)
            option = {"user": "", "reply": "", "attribute_change": ""}

        elif line.startswith("User") or line.startswith("User:"):
            option["user"] = line[5:].strip()
        elif line.startswith("Reply") or line.startswith('**Reply:**'):
            option["reply"] = next(lines).strip()
        elif line.startswith("Attribute Change") or line.startswith('**Attribute Change:**'):
            option["attribute_change"] = line[17:].strip()

    if option:
        dialog["options"].append(option)
    if dialog:
        dialogs.append(dialog)

    with open('emoji_story_23.jsonl', 'a+', encoding="utf-8") as outfile:
        for entry in dialogs:
            json.dump(entry, outfile, ensure_ascii=False)
            outfile.write('\n')

In [ ]:
# 转换为jsonl
for filename in os.listdir('events'):
    if filename.endswith(".txt"):
        try:
            parse_to_jsonl(f'events/{filename}')
        except:
            shutil.move(f'move/{filename}', f'error/{filename}')
            print(filename)